In [ ]:
# credit to Austin

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn import preprocessing
import torch.utils.data as data_utils

In [2]:
%%time
train_transaction = pd.read_csv('newtrain_transaction_200000.csv', index_col='TransactionID')
test_transaction = pd.read_csv('newtest_transaction.csv', index_col='TransactionID')

train_identity = pd.read_csv('newtrain_identity.csv', index_col='TransactionID')
test_identity = pd.read_csv('newtest_identity.csv', index_col='TransactionID')

sample_submission = pd.read_csv('newsample_submission.csv', index_col='TransactionID')

train = train_transaction.merge(train_identity, how='left', left_index=True, right_index=True)
test = test_transaction.merge(test_identity, how='left', left_index=True, right_index=True)

print(train.shape)
print(test.shape)

y_train = train['isFraud'].copy()
del train_transaction, train_identity, test_transaction, test_identity

# Drop target, fill in NaNs
X_train = train.drop('isFraud', axis=1)
X_test = test.copy()

del train, test


(80397, 433)
(10000, 433)
Wall time: 12.9 s


In [3]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

D:\anaconda\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [5]:
from sklearn.linear_model import Lasso, LogisticRegression

In [6]:
y_train.value_counts()

0    78281
1     2116
Name: isFraud, dtype: int64

In [7]:
X_test_filter = X_test.copy()
X_train_filter = X_train.copy()
y_test = X_test_filter['isFraud'].copy()
X_test_filter.drop('isFraud', axis = 1, inplace = True)

### drop all the NaN columns
### dropna() - 
for column in X_test_filter.columns:
    if pd.isnull(X_test_filter[column].unique()[0]):
        X_train_filter.drop([column], axis = 1, inplace=True)
        X_test_filter.drop([column], axis = 1, inplace=True)

In [8]:
# Label Encoding
def label_encoding(X_train, X_test):
    for f in X_train.columns:
        if X_train[f].dtype=='object' or X_test[f].dtype=='object': 
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(X_train[f].values) + list(X_test[f].values))
            X_train[f] = lbl.transform(list(X_train[f].values))
            X_test[f] = lbl.transform(list(X_test[f].values))
    return X_train, X_test


In [9]:
X_train_filter_, X_test_filter_ = label_encoding(X_train_filter, X_test_filter)

In [10]:
### fillna process
"""
- mean
- mode
- 0
"""
def fill_na_df(X_train, X_test):
    for col_name in X_train.columns: 
        X_train[col_name].fillna(X_train[col_name].mode()[0], inplace=True)
        X_test[col_name].fillna(X_test[col_name].mode()[0], inplace=True)

In [11]:
X_train_filter_fill = X_train_filter_.copy()
X_test_filter_fill = X_test_filter_.copy()
fill_na_df(X_train_filter_fill, X_test_filter_fill)

In [12]:
X_train_filter_fill.shape

(80397, 225)

In [13]:
X_test_filter_fill.head()

,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,...,V333,V334,V335,V336,V337,V338,V339,id_01,id_12,id_13
TransactionID,,,,,,,,,,,,,,,,,,,,,
2987022,86786,50.000,1,1724,583.0,150.0,4,226.0,1,299.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-15.0,1,14.0
2987100,88169,100.000,1,16659,170.0,150.0,4,226.0,1,330.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,52.0
2987104,88208,100.000,1,16659,170.0,150.0,4,226.0,1,330.0,...,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,52.0
2987148,88842,30.000,1,14858,558.0,150.0,4,226.0,2,325.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5.0,1,49.0
2987288,90986,155.521,0,16578,545.0,185.0,4,226.0,1,299.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-20.0,1,49.0


In [14]:
X_test_filter_fill.ProductCD.unique()

array([1, 0, 2, 3], dtype=int64)

In [15]:
from sklearn.decomposition import PCA

def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

# X_train_filter_pca = clean_dataset(X_train_filter_)
# X_test_filter_pca = clean_dataset(X_test_filter_)

# pca = PCA(n_components=180)
# pca.fit(X_train_filter_fill)
# X_train_filter_pca = pca.transform(X_train_filter_fill)
# X_test_filter_pca = pca.transform(X_test_filter_fill)
X_train_filter_pca = X_train_filter_fill
X_test_filter_pca = X_test_filter_fill

In [16]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


In [17]:
# %%time
# X_train_filter_fill = reduce_mem_usage(X_train_filter_fill)
# X_test_filter_fill = reduce_mem_usage(X_test_filter_fill)

In [18]:
%%time
X_train_filter_pca = reduce_mem_usage(pd.DataFrame(X_train_filter_pca))
X_test_filter_pca = reduce_mem_usage(pd.DataFrame(X_test_filter_pca))

Memory usage of dataframe is 138.62 MB
Memory usage after optimization is: 37.19 MB
Decreased by 73.2%
Memory usage of dataframe is 17.24 MB
Memory usage after optimization is: 4.72 MB
Decreased by 72.6%
Wall time: 11.2 s


In [19]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

sc = StandardScaler()
X_train_filter_pca = sc.fit_transform(X_train_filter_pca)
# mm=MinMaxScaler()
# X_train_filter_pca=mm.fit_transform(X_train_filter_pca)
X_test_filter_pca = sc.fit_transform(X_test_filter_pca)

X_train2, X_test2, Y_train2, Y_test2 = train_test_split(X_train_filter_pca, y_train, test_size=0.1, random_state=1)

In [20]:
import torch, torch.nn as nn, torch.nn.functional as F
from torch.autograd import Variable

class FraudNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(225, 400)
        self.fc2 = nn.Linear(400,50 )
        self.fc3 = nn.Linear(50, 30)
        self.fc4 = nn.Linear(30, 40)
        self.fc5 = nn.Linear(40, 1)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.dropout(x, p=0.25)
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.sigmoid(self.fc5(x))
        return x
net = FraudNet().double()
net

FraudNet(
  (fc1): Linear(in_features=225, out_features=400, bias=True)
  (fc2): Linear(in_features=400, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=30, bias=True)
  (fc4): Linear(in_features=30, out_features=40, bias=True)
  (fc5): Linear(in_features=40, out_features=1, bias=True)
)

In [21]:
import torch.utils.data as data_utils
from sklearn.metrics import roc_auc_score

X_train = np.array(X_train_filter_pca)
Y_train = np.array(y_train)
X_train = torch.from_numpy(X_train)
Y_train = torch.from_numpy(Y_train).double()


print(X_train.shape[0])
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.002)
training_epochs = 10
minibatch_size = 100
train = data_utils.TensorDataset(X_train, Y_train)
train_loader = data_utils.DataLoader(train, batch_size=minibatch_size, shuffle=True)
for i in range(training_epochs):
    for b, data in enumerate(train_loader):
        inputs, labels = data
        y_pred = net(inputs.double())
        loss = loss_fn(y_pred, labels)
        y_pred1=y_pred
        labels1=labels

#         print('Epochs: {}, batch: {} loss: {}'.format(i, b, loss))
#         print('ROC AUC {}'.format(roc_auc_score(labels1.detach().numpy(), y_pred1.detach().numpy())))
        #reset gradients
        optimizer.zero_grad()

        # backward pass
        loss.backward()

        # update weights
        optimizer.step()
        
y_pred_total = net(X_train.double())
print('ROC AUC {}'.format(roc_auc_score(Y_train.numpy(), y_pred_total.detach().numpy())))

80397


D:\anaconda\lib\site-packages\torch\nn\functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
D:\anaconda\lib\site-packages\torch\nn\modules\loss.py:516: UserWarning: Using a target size (torch.Size([100])) that is different to the input size (torch.Size([100, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
D:\anaconda\lib\site-packages\torch\nn\modules\loss.py:516: UserWarning: Using a target size (torch.Size([97])) that is different to the input size (torch.Size([97, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


ROC AUC 0.9192060899600969


In [22]:
from sklearn.metrics import roc_auc_score

print(X_test_filter_pca[:10])
print(X_test_filter_pca.shape)
X_test = np.array(X_test_filter_pca)
# Y_test= np.array(Y_test2)
X_test = torch.from_numpy(X_test)
print(X_test[:10])
print(X_test.shape)
# Y_test = torch.from_numpy(Y_test).double()
# test = data_utils.TensorDataset(X_test, Y_test)
# test_loader = data_utils.DataLoader(test, batch_size=minibatch_size, shuffle=True)
# correct = 0
# total = 0
# with torch.no_grad():
#     for data in test_loader:
#         inputs, labels = data
#         outputs = net(inputs.double())
# #         print('ROC AUC {}'.format(roc_auc_score(labels.detach().numpy(), outputs.detach().numpy())))
#         _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         correct += (predicted.double() == labels).sum().item()

# print('Accuracy of the network on the {} inputs: {}'.format(
#     X_test.shape[0], 100 * correct/total))
# y_test_pred = net(X_test.double())
# print('ROC AUC {}'.format(roc_auc_score(Y_test.numpy(), y_test_pred.detach().numpy())))

y_text_pred=net(X_test.double())
sample_submission['isFraud'] = y_text_pred.detach().numpy()
print(y_text_pred.detach().numpy()[:10])
print(y_text_pred.detach().numpy().shape)
sample_submission.to_csv('newsample_submission.csv')

[[-1.27319546e+00 -3.32345743e-01 -1.58006105e-03 ... -3.51091423e-01
   4.12477971e-01 -3.10013988e+00]
 [-1.27290789e+00  1.74159056e-01 -1.58006105e-03 ...  7.11319581e-01
   4.12477971e-01  3.05484508e-01]
 [-1.27289978e+00  1.74159056e-01 -1.58006105e-03 ...  7.11319581e-01
  -2.42437190e+00  3.05484508e-01]
 ...
 [-1.27203851e+00  1.44042105e+00  9.85958097e-01 ...  3.57182580e-01
   4.12477971e-01  3.05484508e-01]
 [-1.27197551e+00 -4.33646702e-01 -1.58006105e-03 ...  7.11319581e-01
   4.12477971e-01  3.05484508e-01]
 [-1.27167816e+00 -7.67939869e-01  1.97349626e+00 ...  7.11319581e-01
  -2.42437190e+00  3.05484508e-01]]
(10000, 225)
tensor([[-1.2732e+00, -3.3235e-01, -1.5801e-03,  ..., -3.5109e-01,
          4.1248e-01, -3.1001e+00],
        [-1.2729e+00,  1.7416e-01, -1.5801e-03,  ...,  7.1132e-01,
          4.1248e-01,  3.0548e-01],
        [-1.2729e+00,  1.7416e-01, -1.5801e-03,  ...,  7.1132e-01,
         -2.4244e+00,  3.0548e-01],
        ...,
        [-1.2720e+00,  1.4404

D:\anaconda\lib\site-packages\torch\nn\functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


[[2.18012146e-04]
 [4.07336017e-05]
 [1.53952117e-04]
 [9.42048440e-06]
 [1.16704191e-02]
 [8.84874237e-02]
 [4.07693578e-02]
 [7.93544456e-02]
 [5.12722736e-04]
 [1.22778391e-06]]
(10000, 1)
